In [40]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from unet import create_unet_model
from metrics import psnr

Load data

In [36]:
input_folder = "./images"

image_files = [f for f in os.listdir(input_folder) if f.endswith(".png") or f.endswith(".jpg")]

images_Y = [cv2.imread(os.path.join(input_folder, img)) for img in image_files]

images_X = [cv2.resize(img, (48, 48), interpolation=cv2.INTER_AREA) for img in images_Y]

images_X = [img / 255.0 for img in images_X]
images_Y_ = [img / 255.0 for img in images_Y]

images_X = np.array(images_X)
images_Y = np.array(images_Y)

X_train, X_test, Y_train, Y_test = train_test_split(images_X, images_Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(Y_train.shape)
print(type(X_train))

(4000, 48, 48, 3)
(4000, 96, 96, 3)
<class 'numpy.ndarray'>


In [37]:
model_unet = create_unet_model()
model_unet.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 48, 48, 3)]          0         []                            
                                                                                                  
 conv2d_85 (Conv2D)          (None, 48, 48, 64)           1792      ['input_6[0][0]']             
                                                                                                  
 conv2d_86 (Conv2D)          (None, 48, 48, 64)           36928     ['conv2d_85[0][0]']           
                                                                                                  
 max_pooling2d_15 (MaxPooli  (None, 24, 24, 64)           0         ['conv2d_86[0][0]']           
 ng2D)                                                                                      

In [41]:
model_unet.compile(optimizer='adam', loss='mean_squared_error', metrics=[psnr])

In [42]:
model_unet.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
  4/100 [>.............................] - ETA: 8:39 - loss: 15626.5439 - psnr: -41.3848

KeyboardInterrupt: 